In [13]:
%load_ext autoreload
%autoreload 2

!pip install "gymnasium[accept-rom-license, atari]"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import tensorflow as tf
from code import update_q_network, polyak_averaging, get_small_dqn, get_standard_dqn, sample_trajectory, ReplayBuffer, ENV_SAMPLER
import numpy as np
import tqdm
import pandas as pd
import os

In [15]:
GAMMA = 0.98
TAU = 0.01
BATCHSIZE = 32
N_BATCHES = 20
N_MULTI_ENVS = 64
EXPLORATION_RATE = 0.2

N_NEW_SAMPLES = 1000
N_SAMPLE_SETS = 3

CHECKPOINT_FREQ = 4
LOG_FREQ = 2

SAVE_TO_PATH = "ckpts/"
LOG_PATH = "logs/{}.json"

os.makedirs(SAVE_TO_PATH, exist_ok= True)
os.makedirs(LOG_PATH, exist_ok= True)

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

q_net = get_small_dqn()

target_net = tf.keras.models.clone_model(q_net)

env_sampler = ENV_SAMPLER(q_net, N_MULTI_ENVS)
replay_buffer = ReplayBuffer()

checkpoint = tf.train.Checkpoint(q_net = q_net, target_net = target_net)

Test Polyiak Averging

In [17]:
initial = q_net.trainable_variables[0].numpy()
q_net.trainable_variables[0].assign(np.ones_like(initial))
target_net.trainable_variables[0].assign(np.zeros_like(initial))

polyak_averaging(target_net, q_net, 0.2)
target_net.trainable_variables[0].numpy()

array([[[[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]]],


       [[[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]],

        [[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
         [0.

In [18]:
%%timeit

polyak_averaging(target_net, q_net, 0.2)

1.22 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Test Epsiolonsampling

In [ ]:
state = tf.random.normal((20,84,84,3))

sample_trajectory(q_net, state, 0)


(<tf.Tensor: shape=(20,), dtype=int64, numpy=array([1, 2, 3, 1, 1, 2, 3, 3, 3, 2, 1, 1, 0, 3, 3, 2, 0, 1, 2, 2])>,
 <tf.Tensor: shape=(20,), dtype=float32, numpy=
 array([0.92534614, 0.94207   , 0.93781364, 0.88558877, 0.9155592 ,
        0.95336103, 0.9992751 , 0.8617457 , 0.9306098 , 0.94707036,
        0.9054837 , 0.95591795, 0.92525256, 0.90672493, 0.8282039 ,
        0.89451504, 0.8877164 , 0.9610511 , 0.90063226, 0.9715848 ],
       dtype=float32)>)

In [40]:
a = env_sampler.measure_model_perforamnce(0.999, target_net)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_31569/245931695.py", line 1, in <module>
    a = env_sampler.measure_model_perforamnce(0.999, target_net)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/felix/Desktop/DEEPLR/Week4/env_sampler.py", line 91, in measure_model_perforamnce
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/site-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/site-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py", line 894, in __call__
    result = self._call(*args, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/site-

In [41]:
j = [(1,2,3,),(4,5,6),(7,8,9)]
a,b,c = zip(*j)
a,b,c

[autoreload of replay_buffer failed: Traceback (most recent call last):
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/felix/anaconda3/envs/tf/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/felix/Desktop/DEEPLR/Week4/replay_buffer.py", line 4, in <module>
    from Week4.code import preprocess_all, preprocess_obersvation
ModuleNotFoundError: No module named 'Week4'
]
[autoreload of env_sampler failed: Traceba

((1, 4, 7), (2, 5, 8), (3, 6, 9))

In [7]:
for _ in tqdm.tqdm(range(N_SAMPLE_SETS)):

    new_samples = env_sampler.sample(N_NEW_SAMPLES, epsilon = 1)
    replay_buffer.add_new_trajectory(new_samples)

100%|██████████| 3/3 [00:09<00:00,  3.20s/it]


In [8]:
%%timeit
ds = replay_buffer.generate_tf_dataset(N_BATCHES, BATCHSIZE)

89 ms ± 4.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
ds = replay_buffer.generate_tf_dataset(N_BATCHES, BATCHSIZE)

for _ in ds:
    
    pass

2023-06-10 23:27:35.852651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [640]
	 [[{{node Placeholder/_3}}]]


In [10]:
for data in tqdm.tqdm(ds):
    
    update_q_network(data, q_net, target_net, optimizer, GAMMA)


100%|██████████| 20/20 [00:09<00:00,  2.03it/s]


In [11]:
results = env_sampler.measure_model_perforamnce(100, GAMMA, target_net)
names = ["average_rewards", "average_termination", "average_q_val", "average_target_q_val", "l2_diff",]
results = pd.Series(results, names)

Eval Model: 100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


In [12]:
outer_steps = 0


while True:
    
    outer_steps +=1
    
    ds = replay_buffer.generate_tf_dataset(N_BATCHES, BATCHSIZE)
    
    bar = tqdm.tqdm(ds)
    
    losses = []

    bar = tqdm.tqdm(ds)

    for batch in bar:

        loss = update_q_network(data, q_net, target_net, optimizer, GAMMA)
        
        
        losses.append(loss)
        mean = float(np.mean(losses))
        bar.set_description(f"Loss {mean:.6f}")

    new_samples = env_sampler.sample(N_NEW_SAMPLES, epsilon=1)
    replay_buffer.add_new_trajectory(new_samples)
    polyak_averaging(target_net, q_net, TAU)
    
    
    if outer_steps % LOG_FREQ == 0:
        
        print("Logging Model Metrics")
        
        results = env_sampler.measure_model_perforamnce(100, GAMMA, target_net)
        names = ["average_rewards", "average_termination", "average_q_val", "average_target_q_val", "l2_diff",]
        results = pd.Series(results, names)
        
        results["average_loss"] = np.mean(losses)
        
        results.to_json(LOG_PATH.format(outer_steps))
        
        print(results)
        
        
    if outer_steps % CHECKPOINT_FREQ == 0:
        
        print("Saving Checkpoint")
        
        checkpoint.save(SAVE_TO_PATH)
        
        


  0%|          | 0/20 [00:00<?, ?it/s]
2023-06-10 23:28:27.490858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [640]
	 [[{{node Placeholder/_3}}]]
Loss 0.010932: 100%|██████████| 20/20 [00:08<00:00,  2.47it/s]


Logging Model Metrics


Eval Model: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


average_rewards         0.000000
average_termination          NaN
average_q_val           1.000000
average_target_q_val    2.350118
l2_diff                 1.350122
average_loss            0.010932
dtype: float64
Saving Checkpoint


  0%|          | 0/20 [00:00<?, ?it/s]
2023-06-10 23:29:24.509380: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [640]
	 [[{{node Placeholder/_2}}]]
  0%|          | 0/20 [00:00<?, ?it/s]
2023-06-10 23:29:36.063710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [640]
	 [[{{node Placeholder/_2}}]]
Loss 0.000375:  45%|████▌     | 9/20 [00:04<00:05,  2.07it/s]


KeyboardInterrupt: 